![bse_logo_textminingcourse](https://bse.eu/sites/default/files/bse_logo_small.png)

#  Introduction to Text Mining and Natural Language Processing
## Homework 1: Research Project about hotel prices on Booking.com

GROUP 11: Luis Francisco Alvarez Poli, Vanessa Kromm, Clarice Mottet

In [2]:
import json
import numpy as np
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
from selenium import webdriver
import os
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
import requests


In [3]:
def ffx_preferences(dfolder, download=False):
    '''
    Sets the preferences of the firefox browser: download path.
    '''
    profile = webdriver.FirefoxProfile()
    # set download folder:
    profile.set_preference("browser.download.dir", dfolder)
    profile.set_preference("browser.download.folderList", 2)
    profile.set_preference("browser.download.manager.showWhenStarting", False)
    profile.set_preference("browser.helperApps.neverAsk.saveToDisk",
                           "application/msword,application/rtf, application/csv,text/csv,image/png ,image/jpeg, application/pdf, text/html,text/plain,application/octet-stream")


    # this allows to download pdfs automatically
    if download:
        profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf,application/x-pdf")
        profile.set_preference("pdfjs.disabled", True)

    options = Options()
    options.profile = profile
    return options


def start_up(link, dfolder, geko_path, firefox_binary_path, download=True):
    os.makedirs(dfolder, exist_ok=True)

    options = ffx_preferences(dfolder, download)
    options.binary_location = firefox_binary_path  # Set Firefox binary location
    service = Service(geko_path)
    browser = webdriver.Firefox(service=service, options=options)
    
    # Enter the website address here
    browser.get(link)
    
    time.sleep(5)  
    return browser


def check_and_click(browser, xpath, type):
    '''
    Function that checks whether the object is clickable and, if so, clicks on
    it. If not, waits one second and tries again.
    '''
    ck = False
    ss = 0
    while ck == False:
        ck = check_obscures(browser, xpath, type)
        time.sleep(1)
        ss += 1
        if ss == 15:
            # warn_sound()
            # return NoSuchElementException
            ck = True
            # browser.quit()

def check_obscures(browser, xpath, type):
    '''
    Function that checks whether the object is being "obscured" by any element so
    that it is not clickable. Important: if True, the object is going to be clicked!
    '''
    try:
        if type == "xpath":
            browser.find_element('xpath',xpath).click()
        elif type == "id":
            browser.find_element('id',xpath).click()
        elif type == "css":
            browser.find_element('css selector',xpath).click()
        elif type == "class":
            browser.find_element('class name',xpath).click()
        elif type == "link":
            browser.find_element('link text',xpath).click()
    except (ElementClickInterceptedException, NoSuchElementException, StaleElementReferenceException) as e:
        print(e)
        return False
    return True

In [4]:
# lets open booking:

dfolder='C:/Users/vanes/Downloads'
geko_path='C:/Users/vanes/Desktop/BSE/Term 2/Introduction to Text Mining and NLP/TA-Sessions/geckodriver.exe'
link='https://www.booking.com/index.html?lang=en'
firefox_binary_path = r'C:\Program Files\Mozilla Firefox\firefox.exe'  


browser=start_up(dfolder=dfolder,link=link,geko_path=geko_path, firefox_binary_path=firefox_binary_path)

### close cookies and google pop-up

In [5]:
browser.find_element(by='xpath',value='//*[@id="onetrust-accept-btn-handler"]').click()

In [24]:
from selenium import webdriver
from selenium.webdriver.common.by import By
######NOT WORKING YET############
# Locate the iframe element
iframe = browser.find_element(By.XPATH, '/html/body/div[16]/div[2]/div[2]/iframe')

# Switch to the iframe
browser.switch_to.frame(iframe)

# Interact with elements inside the iframe
element_inside_iframe = browser.find_element(By.XPATH, '/html/body/div/div[1]/div/div[1]/div[2]/svg')
element_inside_iframe.click()

# Switch back to the default content
browser.switch_to.default_content()

NoSuchElementException: Message: Unable to locate element: /html/body/div[16]/div[2]/div[2]/iframe; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16


## Input the place and dates

### First we will look at Barcelona for the time of the event

In [7]:
# input the place
def input_place(place):
    browser.find_element(by='xpath',value='//*[@id=":re:"]').click()
    search1 = browser.find_element(by='xpath',value='//*[@id=":re:"]')
    search1.send_keys(place)


In [8]:
# input the dates
def input_dates(from_day, to_day, change_month = 0):
    css='button.ebbedaf8ac:nth-child(2) > span:nth-child(1)'

    browser.find_element('css selector',css).click()

    # click to change the month to May/June
    if change_month == 1:
        browser.find_element(By.XPATH, '/html/body/div[3]/div[2]/div/form/div[1]/div[2]/div/div[2]/div/nav/div[2]/div/div[1]/button/span/span').click()
        browser.find_element(By.XPATH, '/html/body/div[3]/div[2]/div/form/div[1]/div[2]/div/div[2]/div/nav/div[2]/div/div[1]/button[2]').click()
        browser.find_element(By.XPATH, '/html/body/div[3]/div[2]/div/form/div[1]/div[2]/div/div[2]/div/nav/div[2]/div/div[1]/button[2]').click()
        browser.find_element(By.XPATH, '/html/body/div[3]/div[2]/div/form/div[1]/div[2]/div/div[2]/div/nav/div[2]/div/div[1]/button[2]').click()

    # week from Monday to Sunday covering the whole length of Primavera Sound
    path = '//div[@id="calendar-searchboxdatepicker"]//table[@class="eb03f3f27f"]//tbody//td[@class="b80d5adb18"]//span[@class="cf06f772fa"]'

    dates = browser.find_elements('xpath', path)

    for date in dates:
        date_value = date.get_attribute("data-date")
        
        if date_value == from_day:
            date.click()
        elif date_value == to_day:
            date.click()
            break

## Iterate through all pages and hotels to extract the information

In [9]:
# find total number of pages
def get_number_pages(browser):
    '''
    Get the number of pages. 
    '''
    a = browser.find_elements('xpath',
        '/html/body/div[4]/div/div[2]/div/div[2]/div[3]/div[2]/div[2]/div[4]/div[2]/nav/nav/div/div[2]/ol/li[7]/button')
    return(int(a[-1].text))


In [11]:
#print(pages)

def get_information(pages):
    hotel_names = []
    ratings = []
    room_descriptions = []
    prices = []
    location_descriptions = []

    # beach_distances = []
    
    sections = browser.find_elements('xpath', '//div[@class="c066246e13"]')
    for hotel in sections:
        hotel_name = hotel.find_element('xpath', './/div[@class="f6431b446c a15b38c233"]').text

    # for page in range(int(pages)+1):    
    for page in range(int(2)+1):      
        #Print page that it is in 
        print(f'Page: {page + 1}')
        sections = browser.find_elements('xpath', '//div[@class="c066246e13"]')
        for hotel in sections:
            hotel_name = hotel.find_element('xpath', './/div[@class="f6431b446c a15b38c233"]').text
            hotel_names.append(hotel_name)
            # extract ratings
            try:
                rating = hotel.find_element('xpath', './/div[@class="a3b8729ab1 d86cee9b25"]').text
            except:
                rating = np.nan
            ratings.append(rating)    
            # extract prices
            try:
                price = hotel.find_element('xpath','.//span[@class="f6431b446c fbfd7c1165 e84eb96b1f"]').text
            except:
                price = np.nan
            prices.append(price) 
            # extract room_descriptions
            try:
                room_description = hotel.find_element('xpath','.//div[@class="c19beea015"]').text
            except:
                room_description = np.nan
            room_descriptions.append(room_description)
            # extract location_descriptsion
            try:
                location_description = hotel.find_element('xpath', './/div[@class="abf093bdfe ecc6a9ed89"]').text
            except:
                location_description = np.nan  
            location_descriptions.append(location_description)            
            # try:
            #     beach_distance = hotel.find_element('xpath','.//span[@class="abf093bdfe b058f54b9a"]').text
            # except:
            #     beach_distance = np.nan
            # beach_distances.append(beach_distance)   

    df = pd.DataFrame({'hotel_name': hotel_names, 'rating': ratings, 'room_description': room_descriptions, 'price': prices, 'location_descirption': location_descriptions})
    return df

In [12]:
input_place('Barcelona')
input_dates('2024-05-27', '2024-06-02', 1)
# click on "search"
my_xpath='/html/body/div[3]/div[2]/div/form/div[1]/div[4]/button/span'
check_and_click(browser,my_xpath , type='xpath')    

In [13]:
# close "Genius Pop up"
browser.find_element(By.XPATH, '/html/body/div[49]/div/div/div/div[1]/div[1]/div/button').click()
# browser.find_element(By.XPATH, '/html/body/div[48]/div/div/div/div[1]/div[1]/div/button/span/span').click()

In [14]:
pages = get_number_pages(browser)
barcelona_treatment = get_information(pages)

Page: 1
Page: 2
Page: 3


In [20]:
barcelona_treatment

,hotel_name,rating,room_description,price,location_descirption
0,Hotel Turin Barcelona,8.2,Double Room\n1 double bed\nFree cancellation\n...,"€ 1,465","Ciutat Vella, BarcelonaShow on map450 m from c..."
1,Sonder Los Arcos,8.4,King Room\n1 extra-large double bed\nOnly 1 ro...,"€ 1,530","Ciutat Vella, BarcelonaShow on map1 km from ce..."
2,TWO Hotel Barcelona by Axel 4* Sup- Adults Only,8.4,Double or Twin Room\n1 large double bed\nFree ...,"€ 1,354","Eixample, BarcelonaShow on map1.5 km from cent..."
3,Stylish Apartments,9.4,Deluxe Apartment\nEntire apartment • 2 bedroom...,"€ 5,156","Eixample, BarcelonaShow on map0.8 km from cent..."
4,Catalonia Eixample 1864,8.4,Standard Double or Twin Room\nBeds: 1 double o...,"€ 1,329","Eixample, BarcelonaShow on map0.7 km from cent..."
...,...,...,...,...,...
1075,Catalonia Plaza Catalunya,8.5,Double or Twin Room\nBeds: 1 double or 2 singl...,"€ 1,639","Eixample, BarcelonaShow on map150 m from centr..."
1076,Nice apartment BCN two rooms wifi,NaN,One-Bedroom Apartment\nEntire apartment • 1 be...,€ 511,"Sarrià-St. Gervasi, BarcelonaShow on map2 km f..."
1077,Hostal Las Flores Ramblas,8.2,Double or Twin Room\nBeds: 1 double or 2 singl...,"€ 1,313","Ciutat Vella, BarcelonaShow on map0.7 km from ..."
1078,Casa Elliot by Bondia Hotel Group,8.8,Double or Twin Room\nBeds: 1 double or 2 singles,"€ 2,012","Eixample, BarcelonaShow on map0.7 km from cent..."


### now we have to change the date to extract the hotel prices for the week before the event

In [15]:
input_dates('2024-05-20', '2024-05-26')
# click on "search"
my_xpath='/html/body/div[4]/div/div[2]/div/div[1]/div/form/div[1]/div[4]/button/span'
check_and_click(browser,my_xpath , type='xpath') 
pages = get_number_pages(browser)
barcelona_control = get_information(pages)

In [18]:
barcelona_control

,hotel_name,rating,room_description,price,location_descirption
0,CATERINA Liceu,NaN,One-Bedroom Apartment\nEntire apartment • 1 be...,"€ 4,769","Ciutat Vella, BarcelonaShow on map450 m from c..."
1,Calm Apartments,10,Studio with Balcony\n1 double bed\nFree cancel...,"€ 3,145","Gràcia, BarcelonaShow on map1.8 km from centre..."
2,CATERINA House Port Vell,9.3,Apartment\nEntire apartment • 1 bedroom • 1 li...,"€ 4,843","Ciutat Vella, BarcelonaShow on map1.2 km from ..."
3,C211 Barcelona Apartments,8.7,Apartment with Balcony\nEntire apartment • 2 b...,"€ 5,011","Sarrià-St. Gervasi, BarcelonaShow on map2.1 km..."
4,La Terraza de L'Eixample,NaN,One-Bedroom Apartment\nEntire apartment • 1 be...,"€ 4,932","Eixample, BarcelonaShow on map2.1 km from cent..."
...,...,...,...,...,...
70,limehome Barcelona Rector Triadó,9.1,One-Bedroom Apartment (28+ nights)\nEntire apa...,"€ 10,891","Sants-Montjuïc, BarcelonaShow on map2.4 km fro..."
71,Cozy apartament,NaN,"One-Bedroom Apartment\n2 beds (1 double, 1 sof...","€ 2,473","Eixample, BarcelonaShow on map0.8 km from cent..."
72,CATERINA A90,NaN,One-Bedroom Apartment\nEntire apartment • 1 be...,"€ 5,601","Eixample, BarcelonaShow on map1.5 km from cent..."
73,Apartment CP Poblenou,9.0,One-Bedroom Apartment\nEntire apartment • 1 be...,"€ 4,947","Sant Martí, BarcelonaShow on map3.7 km from ce..."


### now we have to change the city to extract the hotel prices for the week before the event for Rome

In [19]:
input_place('Rome')